<h1>Báo cáo bài tập Cài đặt CBC mode và CTR mode</h1> 
Họ và tên : Đàm Ngọc Khánh<br>
MSSV : 20205207<br>


Đây là đoạn code hàm mã hóa với aes sử dụng thư viện pyaes<br>
Với đầu vào là key, bản rõ và iv <br>
Giải thích : <br>
- Đầu tiên, chuyển bản rõ về dạng bytes rồi kiểm tra số lượng byte cần padding bằng cách lấy   (16 - số byte của bản rõ chia lấy dư cho 16) rồi thêm số byte cần padding vào cuối dãy byte của bản rõ
- Sau đó chạy vòng lặp từng block 16 byte xor với iv rồi mã hóa, iv sẽ được gán = bản mã hóa của khối trước đó
- Từ đó thu được bản mã

<h3> Hàm mã hóa </h3> 

In [227]:
import pyaes
key = "140b41b22a29beb4061bda66b6747e14"
iv = "4ca00ff4c898d61e1edbf1800618fb28"
def Enc_CBC_Mode(key, plaintext, iv) : 
    plaintext_hex = plaintext.encode()
    #Tính số byte phải padding
    num_padding = 16 - len(plaintext_hex) % 16
    #padding
    plaintext_hex += num_padding * num_padding.to_bytes(1, byteorder='big')
    # chuyển chuỗi hex thành byte
    iv_byte = bytes.fromhex(iv)
    #Mã hóa
    aes = pyaes.AES(bytes.fromhex(key))
    # Khởi tạo chuỗi cipher
    ciphers = b'' 
    for i in range(int(len(plaintext_hex)/16)) :
        #Xor iv với bản rõ
        block_xor_iv = bytes([x ^ y for x, y in zip(plaintext_hex[:16], iv_byte)])
        # mã hóa 
        cipher_byte = aes.encrypt(block_xor_iv)
        ciphers += bytes(cipher_byte)
        # bỏ đi 16 byte đã mã hóa của bản rõ 
        plaintext_hex = plaintext_hex[16:]
        # Gán iv thành bản đã mã hóa để sử dụng làm iv cho khối sau
        iv_byte = cipher_byte
    return ciphers

In [228]:
# Check lại hàm mã hóa sau khi đã tìm được bản rõ ở câu 1 và câu 2
plaintext = "Basic CBC mode encryption needs padding."
cipher = Enc_CBC_Mode(key,plaintext, iv).hex()
print(cipher)

28a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81


<h3> Hàm giải mã để giải câu hỏi </h3>
Giải thích : <br>
- Đầu tiên chuyển hết key, bản mã về dạng chuỗi byte rồi tách 16 byte đầu của cipher làm iv <br>
- Chạy vòng for từng khối 16 byte của cipher giải mã với key cho trước rồi xor với iv <br>
- Gán iv lại = bản mã hóa của khối trước đó để dùng làm iv cho khối sau <br>
- Từ đó thu được bản rõ <br>
- Bản rõ còn padding nên ta cần loại bỏ bằng cách chia 16 lấy dư rồi lấy 16 trừ đi để thu được số byte đã padding, từ đó lấy đến vị trí chưa padding ở bản rõ, ta thu được bản rõ ban đầu <br>

In [220]:
import pyaes
def Dec_CBC_Mode(key, cipher) :
    # Khởi tạo plain text
    plain_text = b"" 
    # Chuyển string hex IV -> byte . IV là 16 byte đầu của cipher tức 32 kí tự đầu của cipher dạng hex string
    iv = bytes.fromhex(cipher[0:32])
    # Chuyển key string hex -> byte
    key_byte = bytes.fromhex(key)
    # Chuyển string hex cipher -> byte
    cipher_bytes = bytes.fromhex(cipher[32:])
    # Tạo 1 hàm aes với key byte
    aes = pyaes.AES(key_byte)
    # Chạy bòng for và giải mã từng khối
    for i in range(int(len(cipher_bytes)/16)) :
        # lấy khối đầu tiên ra
        cipher_byte = cipher_bytes[:16]
        # bỏ khối đầu khỏi khối byte cipher
        cipher_bytes = cipher_bytes[16:]
        # Giải mã
        block = aes.decrypt(cipher_byte)
        # Xor với iv rồi thêm vào chuỗi plaintext
        plain_text += bytes([x ^ y for x, y in zip(block, iv)])
        # Gán iv mới = khối byte cipher vừa xét
        iv = cipher_byte
        
    # Xử lý padding
    return plain_text[:-int(plain_text[-1])]

Trả lời câu hỏi 1 và câu hỏi 2

In [222]:
cbc_key1    = "140b41b22a29beb4061bda66b6747e14"
cbc_cipher1 = "4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81"
cbc_key2    = "140b41b22a29beb4061bda66b6747e14"
cbc_cipher2 = "5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253"
print(Dec_CBC_Mode(cbc_key1, cbc_cipher1))
print(Dec_CBC_Mode(cbc_key2, cbc_cipher2))

b'Basic CBC mode encryption needs padding.'
b'Our implementation uses rand. IV'


<h3>Hàm mã hóa với counter mode</h3>

In [225]:
def Enc_CTR_Mode(key, plaintext, iv) : 
    #Chuyển bản rõ và iv -> string byte
    plaintext_bytes = plaintext.encode()
    iv = bytes.fromhex(iv)
    #Khởi tạo hàm aes với key
    aes = pyaes.AES(bytes.fromhex(key))
    #Khởi tại cipher
    ciphers = b''
    # Chạy vòng lặp từng khối 16 byte của bản rõ để mã hóa
    for i in range(int(len(plaintext_bytes)/16) if len(plaintext_bytes)%16 == 0 else int(len(plaintext_bytes)/16) + 1 ) :
        block = aes.encrypt(iv)
        plaintext_byte = plaintext_bytes[:16]
        plaintext_bytes = plaintext_bytes[16:]
        ciphers += bytes([x ^ y for x, y in zip(block, plaintext_byte)])
        #Gán IV mới = iv + 1
        iv = bytearray(iv)
        for i in reversed(range(len(iv))):
            if iv[i] == 255 :
                iv[i] = 0
            else : 
                iv[i] = iv[i] + 1
                break
        iv = bytes(iv)
    return ciphers   

In [226]:
#Check lại hàm mã hóa sau khi đã tìm được bản rõ ở câu 3 và câu 4
plaintext_test = "CTR mode lets you build a stream cipher from a block cipher."
iv_test = "69dda8455c7dd4254bf353b773304eec"
key_test = "36f18357be4dbd77f050515c73fcf9f2"
print(Enc_CTR_Mode(key_test, plaintext_test, iv_test).hex())

0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329


<h3>Hàm giải mã chế độ counter mode</h3>
Giải thích : <br>
- Ta chuyển key, cipher về dạng string byte sau đó tách 16 byte đầu của cipher làm iv <br>
- Khởi tạo hàm aes với khóa, sau đó chạy vòng lặp chạy các khối với kích thước 16 byte, nếu khối cuối không đủ 16 byte thì có bao nhiêu giải bấy nhiêu, vì không padding mà ! <br>
- Ta sẽ mã hóa từng IV với hàm aes đã tạo với key trước đó rồi tăng IV lên 1 cho khối kế tiếp. Lấy kết quả mã hóa xor với khối 16 byte của bản mã ta sẽ thu được bản rõ <br>

In [217]:
def Dec_CTR_Mode(key, cipher) : 
    # Khởi tạo plain text
    plain_text = b"" 
    # Chuyển string hex IV -> byte . IV là 16 byte đầu của cipher tức 32 kí tự đầu của cipher dạng hex string
    iv = bytes.fromhex(cipher[0:32])
    # Chuyển key string hex -> byte
    key_byte = bytes.fromhex(key)
    # Chuyển string hex cipher -> byte
    cipher_bytes = bytes.fromhex(cipher[32:])
    # Tạo 1 hàm aes với key byte
    aes = pyaes.AES(key_byte)
    # Chạy bòng for và giải mã từng khối, số vòng lặp = số khối xác định được
    for i in range(int(len(cipher_bytes)/16) if len(cipher_bytes)%16 == 0 else int(len(cipher_bytes)/16) + 1 ) :
        # lấy khối đầu tiên ra
        cipher_byte = cipher_bytes[:16]
        # bỏ khối đầu khỏi khối byte cipher
        cipher_bytes = cipher_bytes[16:]
        # mã hóa iv
        block = aes.encrypt(iv)
        # Xor với iv rồi thêm vào chuỗi plaintext
        plain_text += bytes([x ^ y for x, y in zip(block, cipher_byte)])
        # Gán iv mới = iv + 1
        iv = bytearray(iv)
        for i in reversed(range(len(iv))):
            if iv[i] == 255 :
                iv[i] = 0
            else : 
                iv[i] = iv[i] + 1
                break
        iv = bytes(iv)
    return plain_text

b'CTR mode lets you build a stream cipher from a block cipher.'


Trả lời câu hỏi 3 và 4

In [219]:
ctr_key = "36f18357be4dbd77f050515c73fcf9f2"
cipher_3 ="69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329"
cipher_4 ="770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451"
print(Dec_CTR_Mode(ctr_key, cipher_3))
print(Dec_CTR_Mode(ctr_key, cipher_4))

b'CTR mode lets you build a stream cipher from a block cipher.'
b'Always avoid the two time pad!'
